In [2]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.9MB/s 


In [3]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime

In [4]:
df=pd.read_csv("NVIDIA.csv")

In [5]:
sentiment = pd.DataFrame(columns=['compound', 'negative', 'neutral', 'positive'])

In [6]:
analyser = SentimentIntensityAnalyzer()
for i in range(len(df)):
  d=analyser.polarity_scores(df["title"].iloc[i])
  sentiment.loc[i]= [d['compound']] + [d['neg']] + [d['neu']] + [d['pos']]

In [7]:
sentiment

,compound,negative,neutral,positive
0,-0.5267,0.132,0.868,0.000
1,0.3612,0.000,0.444,0.556
2,0.0000,0.000,1.000,0.000
3,0.0000,0.000,1.000,0.000
4,0.7351,0.000,0.744,0.256
...,...,...,...,...
998,0.9993,0.034,0.840,0.127
999,0.9755,0.032,0.863,0.106
1000,0.9990,0.030,0.831,0.139
1001,0.7906,0.060,0.836,0.104


In [10]:
frames = [df, sentiment]

result = pd.concat(frames,axis=1)

In [11]:
result

,Unnamed: 0,title,author,date,score,num_comments,upvotes_ratio,compound,negative,neutral,positive
0,0,I have no experience with gsync but is this no...,Due-Department230,2021-05-29 23:05:08,1,3,1.00,-0.5267,0.132,0.868,0.000
1,3,Ready for retirement.,Z06Chris,2021-05-29 18:51:47,36,20,0.84,0.3612,0.000,0.444,0.556
2,4,New MSI RTX 3070 Suprim X build I finished a c...,lllPrecisionlll,2021-05-29 18:32:34,7,4,0.73,0.0000,0.000,1.000,0.000
3,5,3080 Strix white build,matthewgallo98,2021-05-29 17:39:45,6,10,0.65,0.0000,0.000,1.000,0.000
4,7,Dyst-MMXX1 Ver. 1 with a Gigabyte Vision RTX 3...,MechanicalPencilGirl,2021-05-29 15:28:23,0,10,0.40,0.7351,0.000,0.744,0.256
...,...,...,...,...,...,...,...,...,...,...,...
998,998,"**PsychoMarket Recap - Wednesday, May 19, 2021...",psychotrader00,2021-05-19 20:55:50,19,0,0.96,0.9993,0.034,0.840,0.127
999,999,**This is not an attack. This is not cynicism....,egobamyasi,2021-05-16 22:20:34,253,86,0.93,0.9755,0.032,0.863,0.106
1000,1000,https://preview.redd.it/wck74wobywx61.png?widt...,Donisxb,2021-05-08 17:25:47,13,12,0.81,0.9990,0.030,0.831,0.139
1001,1001,New IPO original price $4-7 dollars but got li...,Donisxb,2021-05-06 07:57:37,22,44,0.88,0.7906,0.060,0.836,0.104


In [15]:
result['date'] = pd.to_datetime(result['date']).dt.date

In [16]:
result

,Unnamed: 0,title,author,date,score,num_comments,upvotes_ratio,compound,negative,neutral,positive
0,0,I have no experience with gsync but is this no...,Due-Department230,2021-05-29,1,3,1.00,-0.5267,0.132,0.868,0.000
1,3,Ready for retirement.,Z06Chris,2021-05-29,36,20,0.84,0.3612,0.000,0.444,0.556
2,4,New MSI RTX 3070 Suprim X build I finished a c...,lllPrecisionlll,2021-05-29,7,4,0.73,0.0000,0.000,1.000,0.000
3,5,3080 Strix white build,matthewgallo98,2021-05-29,6,10,0.65,0.0000,0.000,1.000,0.000
4,7,Dyst-MMXX1 Ver. 1 with a Gigabyte Vision RTX 3...,MechanicalPencilGirl,2021-05-29,0,10,0.40,0.7351,0.000,0.744,0.256
...,...,...,...,...,...,...,...,...,...,...,...
998,998,"**PsychoMarket Recap - Wednesday, May 19, 2021...",psychotrader00,2021-05-19,19,0,0.96,0.9993,0.034,0.840,0.127
999,999,**This is not an attack. This is not cynicism....,egobamyasi,2021-05-16,253,86,0.93,0.9755,0.032,0.863,0.106
1000,1000,https://preview.redd.it/wck74wobywx61.png?widt...,Donisxb,2021-05-08,13,12,0.81,0.9990,0.030,0.831,0.139
1001,1001,New IPO original price $4-7 dollars but got li...,Donisxb,2021-05-06,22,44,0.88,0.7906,0.060,0.836,0.104


In [26]:
k=result["date"].unique()

Index(['date', 'score', 'num_comments', 'upvotes_ratio', 'compound',
       'negative', 'neutral', 'positive'],
      dtype='object')

In [41]:
m=[]
for i in range(len(k)):
  l=result[result["date"] == k[i]].sum()
  m.append([k[i]] + [l.score] + [l.num_comments] + [l.upvotes_ratio] + [l.compound] + [l.negative] + [l.neutral] + [l.positive])

In [42]:
df = pd.DataFrame(m,columns=result.drop(['title','author','Unnamed: 0'],axis=1).columns)

In [53]:
df=df.sort_values("date",ascending=False)

In [56]:
first_time = df["date"].iloc[67]
later_time = df["date"].iloc[0]
(later_time - first_time).days

259

In [129]:
li =[]
li.append(df.iloc[0])
for i in range(len(df)):
  first_time = df["date"].iloc[i]
  later_time = li[-1]["date"]
  for j in range((later_time - first_time).days-1):
    li.append(df.iloc[i-1])
    li[-1]["date"] = df["date"].iloc[i-1]+ datetime.timedelta(days = j)
  li.append(df.iloc[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [131]:
pd.DataFrame(li,columns=result.drop(['title','author','Unnamed: 0'],axis=1).columns)

,date,score,num_comments,upvotes_ratio,compound,negative,neutral,positive
0,2021-05-29,2832,315,13.47,1.2111,0.517,15.092,1.391
0,2021-05-29,2832,315,13.47,1.2111,0.517,15.092,1.391
1,2021-05-28,3429,681,16.03,4.0118,0.635,16.966,1.399
2,2021-05-27,3034,862,17.51,5.8863,0.611,20.223,1.167
3,2021-05-26,2400,856,12.56,5.2287,0.265,14.853,0.882
...,...,...,...,...,...,...,...,...
65,2021-04-21,35,14,0.89,0.7783,0.000,0.924,0.076
58,2020-09-15,58,10,0.94,0.4019,0.000,0.426,0.574
58,2020-09-15,58,10,0.94,0.4019,0.000,0.426,0.574
58,2020-09-16,58,10,0.94,0.4019,0.000,0.426,0.574


In [132]:
df2=pd.DataFrame(li,columns=result.drop(['title','author','Unnamed: 0'],axis=1).columns)
df2.to_csv('NVIDIA_sent.csv')